# Tarea 2b: resumen de texto abstracto

En este cuaderno, gestiona los desafíos que surgen al resumir documentos grandes: el texto introducido puede superar la longitud del contexto del modelo, generar salidas alucinantes o provocar errores por falta de memoria.

Para mitigar estos problemas, este cuaderno muestra una arquitectura que utiliza la fragmentación y el encadenamiento de peticiones con el marco de trabajo [LangChain](https://python.langchain.com/docs/get_started/introduction.html), un conjunto de herramientas que permite a las aplicaciones aprovechar los modelos de lenguaje.

Analizará un enfoque que aborda las situaciones en las que los documentos de usuario superan los límites de los tokens. La fragmentación divide los documentos en segmentos por debajo de los umbrales de longitud del contexto antes de introducirlos de manera secuencial en los modelos. Esto encadena las peticiones en los fragmentos, lo que conserva el contexto anterior. Se utiliza este enfoque para resumir transcripciones de llamadas y de reuniones, libros, artículos, publicaciones de blogs, entre otro contenido pertinente.

## Tarea 2b.1: configuración del entorno

En esta tarea, establecerá el entorno.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tarea 2b.2: resumir un texto largo 

### Configurar LangChain con Boto3

En esta tarea, debe especificar el LLM para la clase LangChain Bedrock y puede pasar argumentos para hacer inferencias.

In [ ]:
# model configuration
from langchain_aws import BedrockLLM
modelId = "amazon.titan-text-premier-v1:0"
llm = BedrockLLM(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 2048,
        "temperature": 0,
        "topP": 1
    },
    client=bedrock_client
)

## Tarea 2b.3: cargar un archivo de texto con muchos tokens

En esta tarea, puede buscar un archivo de texto de la [carta del CEO de Amazon a los accionistas en 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) en el directorio de cartas. En la siguiente celda, se carga el archivo de texto y se cuenta la cantidad de tokens. Verá una advertencia que indicará que la cantidad de tokens del archivo de texto supera la cantidad máxima de tokens de este modelo.

In [ ]:
#get tokens
shareholder_letter = "../letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** Puede ignorar las advertencias sin problema y pasar a la siguiente celda.

## Tarea 2b.4: dividir el texto largo en fragmentos

En esta tarea, se divide el texto en fragmentos más pequeños porque es demasiado largo para caber en la petición. `RecursiveCharacterTextSplitter` en LangChain permite dividir el texto largo en fragmentos de forma recursiva hasta que el tamaño de cada fragmento sea inferior a `chunk_size`. Un texto se separa con `separators=["\n\n", "\n"]` en fragmentos, lo que evita que se separe cada párrafo en múltiples fragmentos.

Si se usan 6000 caracteres por fragmento, podrá obtener resúmenes de cada parte por separado. La cantidad de tokens, o partes de palabras, de un fragmento depende del texto.

In [ ]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

## Tarea 2b.5: resumir fragmentos y combinarlos

En esta tarea, en el supuesto que la cantidad de tokens sea la misma que la de los demás documentos, puede continuar. Puede usar `load_summarize_chain` de LangChain para resumir el texto. `load_summarize_chain` proporciona tres formas de resumir: `stuff`, `map_reduce` y `refine`.

- `stuff`: coloca todos los fragmentos en una petición. Por lo tanto, esto alcanzaría el límite máximo de tokens.
- `map_reduce`: resume cada fragmento, combina los resúmenes y resume el resumen combinado. Si el resumen combinado es demasiado grande, se genera un error.
- `refine`: resume el primer fragmento y, luego, resume el segundo fragmento con el primer resumen. Se repite el mismo proceso hasta que todos los fragmentos están resumidos.

Tanto map_reduce como refine invocan el LLM varias veces y demoran en obtener el resumen final. Puede probar map_reduce aquí.

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** La siguiente cadena podría demorar en ejecutarse en función de la cantidad de documentos, la cuota de la tasa de solicitudes de Bedrock y los ajustes de reintento configurados.

In [ ]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# print output
print(output['output_text'])

Experimentó con el uso de la fragmentación y el encadenamiento de peticiones con el marco de trabajo LangChain para resumir documentos grandes, además de mitigar los problemas derivados de texto largo introducido.

### Pruébelo usted mismo
- Cambie las peticiones para que se adapten al caso práctico específico y evalúe la salida de los diferentes modelos.
- Juegue con la longitud del token para comprender la latencia y la capacidad de respuesta del servicio.
- Aplique diferentes principios de ingeniería de peticiones para obtener mejores salidas.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno y continúe con la **Tarea 3**.